In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline

import re

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

from __future__ import division

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

#to see which classes have been defined so far
[modname for importer, modname, ispkg in pkgutil.walk_packages(uproot_methods.classes.__path__)]

loc = "angular_distr_el/CH9/ID_ALL_mol_e0_valid/EN_gate/MFPADs_multinew_std"

# #low old binning (wrong?)
# fileRCR = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\R-C3H3F3O_550eV_CR_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
# fileRCL = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\R-C3H3F3O_550eV_CL_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
# fileSCR = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\S-C3H3F3O_550eV_CR_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
# fileSCL = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\S-C3H3F3O_550eV_CL_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")

#low new binning
fileRCR = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\newbins\R-C3H3F3O_550eV_CR_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
fileRCL = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\newbins\R-C3H3F3O_550eV_CL_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
fileSCR = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\newbins\S-C3H3F3O_550eV_CR_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
fileSCL = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\newbins\S-C3H3F3O_550eV_CL_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")

# #high neew binning
# fileRCR = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\newbins\R-C3H3F3O_550eV_CR_newEfield_ALLCH_MFPAD_30_t0_higherbins.root")
# fileSCR = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\newbins\S-C3H3F3O_550eV_CR_newEfield_ALLCH_MFPAD_30_t0_higherbins.root")
# fileRCL = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\newbins\R-C3H3F3O_550eV_CL_newEfield_ALLCH_MFPAD_30_t0_higherbins.root")
# fileSCL = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\newbins\S-C3H3F3O_550eV_CL_newEfield_ALLCH_MFPAD_30_t0_higherbins.root")

k=fileRCR[loc].keys()
d=dict(fileRCR[loc].classes()) # shows the type of each item
dict(fileRCR[loc]); # keys : values
check = fileRCR[loc+"/MFPAD3D_engate_costheta_-1.00_phi_-180"]
type(check)
check._members();

In [ ]:
b=check.numpy()
print(type(b))
b

# Structure of values inside TH2D. MFPAD_xxx is a list

r_norm_nhistRCR = nhistRCR[0] / nhistRCR[0].sum(axis=1)[:,np.newaxis] # r values normalise to 1

phi_nhistRCR = nhistRCR[1][0][0] # phi

costheta_nhistRCR = nhistRCR[1][0][1] # cos(theta)

In [ ]:
def sorting_array(inarray, cosphi, a):
    """
    a is the level according to which values have to be sorted: 1 = cos_light, 2 = phi_light
    Function to sort the input array according to the labels attached from LMF2ROOT. The lables are stored in cosphi during the loaing.
    To load sorting the 72 items according to cos(theta) use level = 1, according to phi level = 2. The first level is a tring, thereofere
    is not possible to sort according to it.
    NOTE: for experimental data the default order is according to phi, civecersa for the theoretical MFPADs.
    """
    cosn=np.array([col[0] for col in cosphi]); #list
    phin=np.array([col[1] for col in cosphi]); #list
    if inarray.ndim>2:
        data = inarray.reshape(72, inarray.shape[1]*inarray.shape[2]).T
        df = pd.DataFrame(
            data=data,
            index=pd.MultiIndex.from_product([phiM, cosM],names=["phi","cos(theta)"]),
            columns=['item {}'.format(i) for i in range(72)])
            
    else:
        cosbin = inarray.shape[1]        
        data = inarray.T
        df = pd.DataFrame(
            data=data,
            index=np.linspace(-1,1,inarray.shape[1]),
            columns=['item {}'.format(i) for i in range(72)])
    
    #sorting according to the cosphi array
    df1=df.T
    df1["cos_light"]=cosn
    df1["phi_light"]=phin
    df1.set_index("cos_light", append=True, inplace=True) #level=1
    df1.set_index("phi_light", append=True, inplace=True) #level=2
    
    #here it transforms the DataBAse into a series in order to sort correctly the cos(theta) column too
    dfnew=df1.stack()
    #level=2 is the standard for experiment, level=1 for theory
    #kind="mergesort" the concept of stable sorting algorithm
    dfnew.sort_index(level=a, inplace=True, kind="mergesort")
    dfnew.reset_index(level=1, drop=True, inplace=True) # removes cos_light
    dfnew.reset_index(level=1, drop=True, inplace=True) # removes phi
    df2=dfnew.T

    if inarray.ndim>2:
        #fast approach with numpy
        MFPAD_temp = df2.values.reshape(inarray.shape[1], inarray.shape[2], 72)
        #Then call transpose to rearrange the order of the axes:
        outarray=MFPAD_temp.transpose(2, 0, 1)
    else:
        outarray=df2.values.reshape(72, inarray.shape[1])
        
    return outarray

def cosphi_func(key, cosphi):
    ctheta_nc = float((str(key).split("costheta_")[1]).split("_phi")[0])
    phi_nc = float((str(key).split("phi_")[1]).split(";")[0])
    # alternative method with import re
    #ctheta_n=re.search("costheta_(.*)_phi", str(key)).group(1)
    cosphi.append((ctheta_nc, phi_nc))
    return cosphi

def shift_func(a, flag=0.):
    if a.shape[0]>12:
        c=a.reshape(12,6)
        flag=1.
        if c[0][0] == c[0][1]:
            b=(c[1][0]-c[0][0])/2
            a=np.add(c,b)
        else:
            b=(c[0][1]-c[0][0])/2
            a=np.add(c,b)
    else:
        if a[0][0] == a[0][1]:
            b=(a[1][0]-a[0][0])/2
            a=np.add(a,b)
        else:
            b=(a[0][1]-a[0][0])/2
            a=np.add(a,b)
    if flag == 1.:
        # a=np.ravel(a)
        a=a.reshape(-1)
    return a

def import_MFPAD(file, MFPAD, cosphi, MFPAD_xy, ctheta, ctheta_c, ctheta_cred, run_MFPAD=0., run_cos=0.):
    """
    Loads the MFPADs and the cos(theta) from the .root files.
    6 inputs + a parameter
    NOTE: MFPAD_xy and ctheta_c have originally +1 dimensions compare to the z values.
    For the sake of iminiut, cos(theta) is centered on the middle of the bins.
    """
    for key, value in file[loc].items():
        filename=loc+"/"+str(key).split(";")[0].replace("b'","")
        if "mfpad3d_engate_costheta" in filename.lower():
            cosphi=cosphi_func(key,cosphi)
            temp=np.array(file[filename].numpy())
            MFPAD.append(temp[0]) # it is a list!
            if run_MFPAD == 0.:
                MFPAD_xy.append((temp[1][0][0] , temp[1][0][1])) # phi cos(theta) from 2D
                run_MFPAD=1.
        elif "cos(theta)" in filename.lower():
            temp=np.array(file[filename].numpy())
            ctheta.append(temp[0]) # it is a list!
            if run_cos == 0.:
                ctheta_c.append(temp[1])
                ctheta_cred.append(np.array((ctheta_c[0][1:] + ctheta_c[0][:-1])/2)) #! reduced of 1 dimension
                run_cos=1.
    return MFPAD, cosphi, MFPAD_xy, ctheta, ctheta_c, ctheta_cred

In [ ]:
# initilialises a list
MFPAD_RCR=[]; ctheta_RCR=[]; ctheta_xaxis_RCR=[]; cosphi_RCR=[];
MFPAD_RCL=[]; ctheta_RCL=[]; ctheta_xaxis_RCL=[]; cosphi_RCL=[];
MFPAD_SCR=[]; ctheta_SCR=[]; ctheta_xaxis_SCR=[]; cosphi_SCR=[];
MFPAD_SCL=[]; ctheta_SCL=[]; ctheta_xaxis_SCL=[]; cosphi_SCL=[];

MFPAD_xy=[];
ctheta_cred=[];

import_MFPAD(fileRCR, MFPAD_RCR, cosphi_RCR, MFPAD_xy, ctheta_RCR, ctheta_xaxis_RCR, ctheta_cred)
import_MFPAD(fileRCL, MFPAD_RCL, cosphi_RCL, MFPAD_xy, ctheta_RCL, ctheta_xaxis_RCL, ctheta_cred)
import_MFPAD(fileSCR, MFPAD_SCR, cosphi_SCR, MFPAD_xy, ctheta_SCR, ctheta_xaxis_SCR, ctheta_cred)
import_MFPAD(fileSCL, MFPAD_SCL, cosphi_SCL, MFPAD_xy, ctheta_SCL, ctheta_xaxis_SCL, ctheta_cred)

# converting into nparray
MFPAD_RCR=np.array(MFPAD_RCR); ctheta_RCR=np.array(ctheta_RCR); ctheta_xaxis_RCR=np.array(ctheta_xaxis_RCR[0]); cosphi_RCR=np.array(cosphi_RCR);
MFPAD_RCL=np.array(MFPAD_RCL); ctheta_RCL=np.array(ctheta_RCL); ctheta_xaxis_RCL=np.array(ctheta_xaxis_RCL[0]); cosphi_RCL=np.array(cosphi_RCL);
MFPAD_SCR=np.array(MFPAD_SCR); ctheta_SCR=np.array(ctheta_SCR); ctheta_xaxis_SCR=np.array(ctheta_xaxis_SCR[0]); cosphi_SCR=np.array(cosphi_SCR);
MFPAD_SCL=np.array(MFPAD_SCL); ctheta_SCL=np.array(ctheta_SCL); ctheta_xaxis_SCL=np.array(ctheta_xaxis_SCL[0]); cosphi_SCL=np.array(cosphi_SCL);

ctheta_cred=ctheta_cred[0];
cosphi=cosphi_RCR;

In [ ]:
# print(ctheta_xaxis_RCR, "\n")
# print(ctheta_cred, "\n")
# print(cosphi.shape[1], "\n")
# print(cosphi[:,1].reshape(12,6), "\n")
print(shift_func(cosphi[:,1].reshape(12,6)), "\n")
print(shift_func(cosphi[:,0].reshape(12,6)), "\n")
print(shift_func(cosphi[:,1]), "\n")
print(shift_func(cosphi[:,0]))

In [ ]:
test=fileRCL[loc+"/cos(theta)_e[0]_costheta_-1.00_phi_-180"]
# test.__dir__()
# print(test._fTsumwx2)
# print(test.variances)
print(test.values)
errc = np.sqrt(test.values)
print(errc)

In [ ]:
plt.scatter(range(len(test.values)),test.values,color='r')
plt.errorbar(range(len(test.values)),test.values, yerr=errc, fmt="o")

In [ ]:
print(MFPAD_xy[0][0].shape)
MFPAD_xy[0][0] #to be reduced like ctheta_c_red

In [ ]:
print(np.array(ctheta_RCR).shape,ctheta_RCR)
print(np.array(ctheta_RCL).shape,ctheta_RCL)

In [ ]:
print(ctheta_RCR[0].shape,ctheta_RCR[0])
print(ctheta_RCL[0].shape,ctheta_RCL[0])

In [ ]:
ctheta_RCR_new=sorting_array(ctheta_RCR,cosphi,1)
ctheta_RCL_new=sorting_array(ctheta_RCL,cosphi,1)

ctheta_RCR_phi=sorting_array(ctheta_RCR,cosphi,2)
ctheta_RCL_phi=sorting_array(ctheta_RCL,cosphi,2)

print(ctheta_RCR==ctheta_RCR_phi)
print(ctheta_RCL==ctheta_RCL_phi)

In [ ]:
#test of similarity

cPECD_norm_std =  np.divide(np.subtract(ctheta_RCR/ctheta_RCR.sum(axis=1)[:,np.newaxis],ctheta_RCL/ctheta_RCL.sum(axis=1)[:,np.newaxis]),np.add(ctheta_RCR/ctheta_RCR.sum(axis=1)[:,np.newaxis],ctheta_RCL/ctheta_RCL.sum(axis=1)[:,np.newaxis]))

cPECD_std =  np.divide(np.subtract(ctheta_RCR,ctheta_RCL),np.add(ctheta_RCR,ctheta_RCL))

errPECD_std = np.sqrt(1/(ctheta_RCR+ctheta_RCL))

In [ ]:
cPECD_norm =  np.divide(np.subtract(ctheta_RCR_new/ctheta_RCR_new.sum(axis=1)[:,np.newaxis],ctheta_RCL_new/ctheta_RCL_new.sum(axis=1)[:,np.newaxis]),np.add(ctheta_RCR_new/ctheta_RCR_new.sum(axis=1)[:,np.newaxis],ctheta_RCL_new/ctheta_RCL_new.sum(axis=1)[:,np.newaxis]))

cPECD =  np.divide(np.subtract(ctheta_RCR_new,ctheta_RCL_new),np.add(ctheta_RCR_new,ctheta_RCL_new))

# from the statistical book var = sqrt(1/Ntot)

errPECD = np.sqrt(1/(ctheta_RCR_new+ctheta_RCL_new))

print(cPECD.shape)
print(cPECD_norm.shape)
print(cPECD_norm[0].shape)
#TRUE if level = 2
print(ctheta_RCL==ctheta_RCL_new)
diff1=ctheta_RCL-ctheta_RCL_new
plt.plot(diff1[0])
print(cPECD_norm_std==cPECD_norm)
diff=cPECD_norm_std-cPECD_norm
# plt.plot(diff[0])

plt.scatter(ctheta_cred,cPECD_norm[0])

In [ ]:
# everything in iminuit is done through the Minuit object, so we import it
from iminuit import Minuit
# we also need a cost function to fit and import the LeastSquares function
from iminuit.cost import LeastSquares
from iminuit.util import describe, make_func_code
import traceback

class LeastSquares:
    """
    Generic least-squares cost function.
    """
    def __init__(self, model, x, y):
        self.model = model  # model predicts y for given x
        self.x = np.array(x)
        self.y = np.array(y)

    def __call__(self, *par):  # we accept a variable number of model parameters
        ym = self.model(self.x, *par)
        chi2 = np.sum((self.y - ym)**2)
        return chi2

# does this inherits from LeastSquare?
class BetterLeastSquares(LeastSquares):
    def __init__(self, model, x, y):
        super().__init__(model, x, y)
        self.func_code = make_func_code(describe(model)[1:])

#NOTE +1 IS NOT NECESSARY!!!
def PECD6(x, b1, b2, b3, b4, b5, b6):
    return ((b1*x + 0.5*b3*(5*x**3-3*x) +0.125*b5*(63*x**5-70*x**3+15*x)) / (1 + 0.5*b2*(3*x**2-1) + 0.125*b4*(35*x**4-30**2+3) + 0.0625*b6*(231*x**6-315*x**4+105*x**2-5)))

x_data = ctheta_cred
y_data = cPECD_norm[0]

# class iminuit.cost.LeastSquares(x, y, yerror, model, loss='linear', verbose=0)
# Use this if you have data of the form (x, y +/- yerror).
lsq = BetterLeastSquares(PECD6, x_data, y_data,)

m = Minuit(lsq, errordef=Minuit.LEAST_SQUARES, limit_b1=(-1, 1), limit_b2=(-1, 1), limit_b3=(-1, 1), limit_b4=(-1, 1), limit_b5=(-1, 1), limit_b6=(-1, 1)) # initial values are 0 by default
# m = Minuit(lsq, error_b1=0.01, error_b2=0.01, limit=((-1, 1), (-1, 1)), name=("b1", "b2"), pedantic=False)
m.migrad()
# print(m.params)

In [ ]:
# modified with error
# everything in iminuit is done through the Minuit object, so we import it
from iminuit import Minuit
# we also need a cost function to fit and import the LeastSquares function
from iminuit.cost import LeastSquares
from iminuit.util import describe, make_func_code
import traceback

class LeastSquares:
    """
    Generic least-squares cost function with error.
    """
    def __init__(self, model, x, y, err):
        self.model = model  # model predicts y for given x
        self.x = np.array(x)
        self.y = np.array(y)
        self.err = np.array(err)
        # self.err = err

    def __call__(self, *par):  # we accept a variable number of model parameters
        ym = self.model(self.x, *par)
        chi2 = np.sum((self.y - ym)**2/(self.err**2))
        return chi2

# does this inherits from LeastSquare?
class BetterLeastSquares(LeastSquares):
    def __init__(self, model, x, y, err):
        super().__init__(model, x, y, err)
        self.func_code = make_func_code(describe(model)[1:])

#NOTE +1 IS NOT NECESSARY!!!
def PECD6(x, b1, b2, b3, b4, b5, b6):
    return ((b1*x + 0.5*b3*(5*x**3-3*x) +0.125*b5*(63*x**5-70*x**3+15*x)) / (1 + 0.5*b2*(3*x**2-1) + 0.125*b4*(35*x**4-30**2+3) + 0.0625*b6*(231*x**6-315*x**4+105*x**2-5)))

x_data = ctheta_cred
y_data = cPECD_norm[0]
y_err = errPECD[0]

# class iminuit.cost.LeastSquares(x, y, yerror, model, loss='linear', verbose=0)
# Use this if you have data of the form (x, y +/- yerror).
lsq = BetterLeastSquares(PECD6, x_data, y_data, y_err)

# m = Minuit(lsq, errordef=Minuit.LEAST_SQUARES,limit_b1=(-1, 1), limit_b2=(-1, 1), limit_b3=(-1, 1), limit_b4=(-1, 1), limit_b5=(-1, 1), limit_b6=(-1, 1)) # initial values are 0 by default
m = Minuit(lsq, errordef=Minuit.LEAST_SQUARES, limit_b1=(-1, 1), limit_b2=(-1, 1))
m.migrad()
m.hesse() # run covariance estimator
# print(m.params)

In [ ]:
from scipy.interpolate import make_interp_spline, BSpline
x_data_new = np.linspace(x_data.min(), x_data.max(), 300)
bspl = make_interp_spline(x_data, PECD6(x_data, *m.values.values()), k=5)
plt.plot(x_data_new, bspl(x_data_new))
plt.scatter(ctheta_cred,cPECD_norm[0],color='r')
plt.errorbar(ctheta_cred,cPECD_norm[0], yerr=errPECD[0], fmt="o")
plt.show()
for p in m.parameters:
    print("{} = {:2.3e} +/- {:2.3e}".format(p, m.values[p], m.errors[p]))

In [ ]:
# modified with error
# everything in iminuit is done through the Minuit object, so we import it
from iminuit import Minuit
# we also need a cost function to fit and import the LeastSquares function
from iminuit.cost import LeastSquares
from iminuit.util import describe, make_func_code
import traceback

class LeastSquares:
    """
    Generic least-squares cost function with error.
    """
    def __init__(self, model, x, y, err):
        self.model = model  # model predicts y for given x
        self.x = np.array(x)
        self.y = np.array(y)
        self.err = np.array(err)
        # self.err = err

    def __call__(self, *par):  # we accept a variable number of model parameters
        ym = self.model(self.x, *par)
        chi2 = np.sum((self.y - ym)**2/(self.err**2))
        return chi2

# does this inherits from LeastSquare?
class BetterLeastSquares(LeastSquares):
    def __init__(self, model, x, y, err):
        super().__init__(model, x, y, err)
        self.func_code = make_func_code(describe(model)[1:])

# param_matrix=np.zeros((6,72)) #nparray initialization
param_matrix = [[0 for i in range(72)] for j in range(6)] #i col, j row

fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex=True , sharey=True)
fig.subplots_adjust(hspace = .5, wspace=.5)
custom_xlim = (-1, 1)
custom_ylim = (-0.2,0.2)

for i in range(72): #fixe because of the photon directions, equal to range(len(cosphi))
    #print(i)
    x_data = ctheta_cred
    y_data = cPECD_norm[i]
    y_err = errPECD[i]

    lsq = BetterLeastSquares(PECD6, x_data, y_data, y_err)

    m = Minuit(lsq, errordef=Minuit.LEAST_SQUARES, limit_b1=(-1, 1), limit_b2=(-1, 1), limit_b3=(-1, 1), limit_b4=(-1, 1), limit_b5=(-1, 1), limit_b6=(-1, 1)) # initial values are 0 by default
    # m = Minuit(lsq, error_b1=0.01, error_b2=0.01, limit=((-1, 1), (-1, 1)), name=("b1", "b2"), pedantic=False)
    m.migrad() # run optimiser
    m.hesse() # run covariance estimator
    # print(m.params)
    x_data_new = np.linspace(x_data.min(), x_data.max(), 300)
    bspl = make_interp_spline(x_data, PECD6(x_data, *m.values.values()), k=5)
    
    ax = fig.add_subplot(6, 12, i+1)
    ax.plot(x_data_new, bspl(x_data_new))
    ax.scatter(ctheta_cred,y_data,color='r')
    ax.set_xlim(custom_xlim)
    ax.set_ylim(custom_ylim)
    ax.errorbar(ctheta_cred,y_data, yerr=y_err, fmt="o")
    # ax.axis('off')
    for j,p in zip(range(len(m.parameters)),m.parameters): # len = 6
        param_matrix[j][i] = ((m.values[p], m.errors[p]))
        # param_matrix[i][j] = (m.values[p], m.errors[p])

fig.tight_layout()
plt.show()
    # for p in m.parameters:
    #     print("{} = {:2.3e} +/- {:2.3e}".format(p, m.values[p], m.errors[p]))

In [ ]:
import pprint
pprint.pprint(param_matrix)
print(len(m.parameters))

In [ ]:
test2=[col[0] for col in param_matrix]; #list
b1=[col[0] for col in test2];

In [ ]:
param_matrix=np.array(param_matrix) #numpy tranformation!!!
b1 = param_matrix[0,:,0]
b2 = param_matrix[1,:,0]
print(b1[0],b1.shape, b2[0], b2.shape)
plt.plot(param_matrix[0,:,0])

In [ ]:
fig,axes = plt.subplots(1,6, figsize=(50, 7.5))
for i in range(6):
    # ax[0,1] = fig.add_subplot(1, 2, 1)
    axes[i].contourf(shift_func(cosphi[:,1].reshape(12,6)) , shift_func(cosphi[:,0].reshape(12,6)) , param_matrix[i,:,0].reshape(12,6) , np.arange(-1., 1., .005) , extend='both', cmap='seismic')
    # axes[i].contourf(shift_func(cosphi[:,0].reshape(12,6)),shift_func(cosphi[:,1].reshape(12,6)),param_matrix[i,:,0].reshape(6,12).T)
plt.show()

In [ ]:
import matplotlib as mpl
from matplotlib import cm

#the following converts a cmap into plotly language 
magma_cmap = mpl.cm.get_cmap('magma')
seismic_cmap = mpl.cm.get_cmap('seismic')

magma_rgb = []
seismic_rgb = []
norm = mpl.colors.Normalize(vmin=0, vmax=255)

for i in range(0, 255):
       k = mpl.colors.colorConverter.to_rgb(magma_cmap(norm(i)))
       magma_rgb.append(k)

for i in range(0, 255):
       k = mpl.colors.colorConverter.to_rgb(seismic_cmap(norm(i)))
       seismic_rgb.append(k)

def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []

    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])

    return pl_colorscale

Magma_r = matplotlib_to_plotly(magma_cmap, 255)
Seismic_r = matplotlib_to_plotly(seismic_cmap, 255)

In [ ]:
go_cos=shift_func(cosphi[:,0]).T;
print(go_cos.shape)
test=go_cos.tolist()
print(test[0], "\n")
print(type(cosphi[:,0]), cosphi[:,0].shape)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

go_cos=shift_func(cosphi[:,0]).tolist();
go_phi=shift_func(cosphi[:,1]).tolist();

fig = make_subplots(rows=1, cols=2,
                    subplot_titles=('Without Smoothing', 'With Smoothing'))

fig.add_trace(go.Contour(z=param_matrix[0,:,0], x=go_phi, y=go_cos, line_smoothing=0, colorscale=Seismic_r, contours=dict(start=-0.2, end=0.2, size=0.04)), 1, 1)
fig.add_trace(go.Contour(z=param_matrix[0,:,0], x=go_phi, y=go_cos, line_smoothing=0.75, colorscale=Seismic_r,contours=dict(start=-0.2, end=0.2, size=0.04)), 1, 2)

fig.update_layout(
    # coloraxis=dict(colorscale=Seismic_r),
    # showlegend=False,
    autosize=False,
    width=900,
    height=450,
    margin=dict(l=5,r=5,b=25,t=25)
    )

fig.show()

In [ ]:
#NOTE: THE SCALE IS FAKE FOR PLOT 2 AND 6!

import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = go.Figure()

fig = make_subplots(rows=6, cols=2, vertical_spacing=0.015,
                    subplot_titles=('Without Smoothing', 'With Smoothing'))

for i in range(len(param_matrix)):
    if i==1 or i==5:
        fig.add_trace(go.Contour(z=param_matrix[i,:,0], x=go_phi, y=go_cos, line_smoothing=0, colorscale=Seismic_r, legendgroup="no", showscale=False, showlegend=False, contours=dict(start=-1, end=1, size=0.5)), i+1, 1)
        fig.add_trace(go.Contour(z=param_matrix[i,:,0], x=go_phi, y=go_cos, line_smoothing=0.85, colorscale=Seismic_r, legendgroup="no", showscale=False, showlegend=False, contours=dict(start=-1, end=1, size=0.5)), i+1, 2)
    else:
        fig.add_trace(go.Contour(z=param_matrix[i,:,0], x=go_phi, y=go_cos, line_smoothing=0, colorscale=Seismic_r, legendgroup="yes", contours=dict(start=-0.2, end=0.2, size=0.04)), i+1, 1)
        fig.add_trace(go.Contour(z=param_matrix[i,:,0], x=go_phi, y=go_cos, line_smoothing=0.85, colorscale=Seismic_r, legendgroup="yes", contours=dict(start=-0.2, end=0.2, size=0.04)), i+1, 2)

fig.update_layout(
    # coloraxis=dict(colorscale=Seismic_r),
    # showlegend=False,
    autosize=False,
    # width=800,
    height=1500,
    margin=dict(l=0,r=0,b=25,t=25)
    )

fig.show()